## TODO Test CNN network
## Change random sampler to Gaussian distribution
## Experiment with https://github.com/soumith/ganhacks hints

In [ ]:
import os,random
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.losses import categorical_crossentropy
from keras import backend as K

from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.datasets import mnist
import matplotlib.pyplot as plt
import random, sys, keras
from keras.models import Model, Sequential
from IPython.display import SVG, display
from keras.utils.vis_utils import model_to_dot
from keras.utils import np_utils
from tqdm import tqdm
from sklearn.utils import shuffle


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0" #On rd010532 0 = Tesla, 1 = Quadro

In [ ]:
def draw_model_on_interactive_session(model):

    display(SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg')))


In [ ]:
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print(np.min(X_train), np.max(X_train))

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


In [ ]:
# Build Generative model ...
opt = Adam(lr=1e-4)

generator = Sequential(name='generator')
generator.add(Dense(3136, input_shape=(100,), activation='relu'))
generator.add(Reshape((56, 56, 1)))
generator.add(Conv2D(50, (3, 3),activation='relu', padding='same'))
generator.add(Conv2D(25, (3, 3), strides=(2, 2), activation='relu', padding='same'))
generator.add(Conv2D(1, (1, 1), activation='sigmoid', padding='same'))

generator.compile(loss='mean_squared_error', optimizer=opt)
generator.summary()
draw_model_on_interactive_session(generator)

In [ ]:
def initialise_generator():
    X_train[0:1000].shape
    y = np.random.normal(0,1,size=[1000,100])

    generator.fit(y, X_train[0:1000], epochs=10, verbose=1)


In [ ]:

# Build Discriminative model ...
dopt = Adam(lr=1e-3)

discriminator = Sequential(name='discriminator')
discriminator.add(Dense(28 * 28 * 1, input_shape=(28, 28, 1)))
discriminator.add(Conv2D(32, (5, 5), strides=(2, 2), padding='same', activation='relu'))
discriminator.add(Conv2D(32, (5, 5), strides=(2, 2), padding='same', activation='relu'))
discriminator.add(Flatten())
discriminator.add(Dense(256, activation='relu'))
discriminator.add(Dense(1, activation='tanh'))
discriminator.compile(loss='binary_crossentropy', optimizer=dopt)
#discriminator.compile(loss=GAN_loss_function, optimizer=dopt)
discriminator.summary()

draw_model_on_interactive_session(discriminator)

In [ ]:
# Freeze weights in the network for when using stacked training
def make_trainable(network, val):
    network.trainable = val
    
    for layer in network.layers:
        layer.trainable = val

    network.compile(loss=network.loss, optimizer=network.optimizer)


In [ ]:
# Build stacked GAN model
# need new loss function here? Maximise entropy between two classes while
#discriminator is trying to reduce it?

GAN = Sequential()
GAN.add(generator)
GAN.add(discriminator)
GAN.compile(loss='binary_crossentropy', optimizer='adam')

GAN.summary()

draw_model_on_interactive_session(GAN)

In [ ]:
losses = {'d':[], 'g':[], 'accuracy':[]}


def plot_loss(losses):   
    
    if ax.lines:
        ax.lines[0].set_data(list(range(len(losses["d"]))), losses["d"])
        ax.lines[1].set_data(list(range(len(losses["g"]))), losses["g"])
        ax.lines[2].set_data(list(range(len(losses["accuracy"]))), losses["accuracy"])
        ax.autoscale(True)
        
    else:        
        ax.plot(list(range(len(losses["d"]))), losses["d"], label='discriminator loss')
        ax.plot(list(range(len(losses["g"]))), losses["g"], label='generator loss')
        ax.plot(list(range(len(losses["accuracy"]))), losses["accuracy"], label='accuracy')
        ax.autoscale()        
        
    ax.relim()
    ax.autoscale_view()
    ax.legend(['Discriminator Loss', 'Generator Loss', 'Discriminator Accuracy'])
    fig.canvas.draw()

    
def plot_gen(n_ex=16,dim=(4,4), figsize=(10,10) ):
    
    noise = np.random.normal(0,1,size=[n_ex,100])
    generated_images = generator.predict(noise)
    
    for i in range(generated_images.shape[0]):
        #plt.subplot(dim[0],dim[1],i+1)
        plt.subplot(dim[0],dim[1],i+1)
        img = generated_images[i,:,:,0]
        plt.imshow(img, cmap='gray')
        plt.axis('off')
    
    plt.show()
    image_fig.canvas.draw()


In [ ]:
def pretrain_discriminator():
    make_trainable(discriminator,True)

    X, y = generate_discriminator_training_batch(30000)
    
    discriminator.fit(X, y, epochs=1, batch_size=250)


In [ ]:
ntrain = 100

def perform_discriminator_training_steps(n_steps=1):
    make_trainable(discriminator,True)

    X, y = generate_discriminator_training_batch(ntrain)

    discriminator_history = 0

    for _ in range(n_steps):
        discriminator_history = discriminator.train_on_batch(X,y)

    losses['d'].append(discriminator_history)

    y_hat = discriminator.predict(X)

    accuracy, n_right, n_total = calculate_discriminator_accuracy(X, y)

    losses['accuracy'].append(accuracy)
    

In [ ]:

def generate_discriminator_training_batch(batch_size):
    trainidx = random.sample(range(0,X_train.shape[0]), int(batch_size/2))
    XT = X_train[trainidx,:,:,:]

    noise_gen = np.random.normal(0,1,size=[int(batch_size/2),100])

    generated_images = generator.predict(noise_gen)
    X = np.concatenate((XT, generated_images))
    n = XT.shape[0]
    y = np.zeros([2*n])
    y[:n] = 1 #real sample = 1
    y[n:] = 0 #generated sample = 0

    #X, y = shuffle(X, y)
    
    return X, y

In [ ]:
def calculate_discriminator_accuracy(input_images, labels):
    
    def measure_accuracy(labels, predicted):
        # Measure accuracy of pre-trained discriminator network
        predicted = np.around(predicted)
        total_correct = sum(predicted == labels)[0]
        accuracy = total_correct*100.0/len(labels)
        return accuracy, total_correct, len(labels)

    
    y_predicted = discriminator.predict(input_images)

    accuracy, n_right, n_total = measure_accuracy(labels, y_predicted)
    return accuracy, n_right, n_total


In [ ]:
 # train Generator-Discriminator stack on input noise to non-generated output class
def perform_generator_training_steps(n_steps=1):
    noise_tr = np.random.normal(0,1,size=[ntrain, 100])
    y2 = np.zeros([ntrain])
    y2[:] = 1 #pretend that the generated images are real

    make_trainable(discriminator, False)

    #GAN.layers[2].layers[6].trainable
    #gan_history = GAN.fit(noise_tr, y2, epochs=10, shuffle=True )
    gan_history = 0
    for _ in range(n_steps):
        gan_history = GAN.train_on_batch(noise_tr, y2)
        
    losses['g'].append(gan_history)


In [ ]:
initialise_generator()

In [ ]:
pretrain_discriminator()

In [ ]:
perform_generator_training_steps(50)

In [ ]:
perform_discriminator_training_steps(1)

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(1, 1)

image_fig, image_ax = plt.subplots(1, 1)

for _ in range(6000):
    perform_discriminator_training_steps(1)
    perform_generator_training_steps(1)

    fig
    plot_loss(losses)
        
    image_fig
    plot_gen()

In [ ]:
GAN.predict(np.random.normal(0,1,size=[1,100]))

In [ ]:
discriminator.predict(np.array([X_train[0]]))